Data Wrangling Notebook for SY Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [35]:
import pandas as pd
from pandas import Series
import numpy as np
import re
import uuid 

Import SY Data

In [36]:
sy_data = pd.read_csv("./../Original_Data/futres_sy_original.csv")

Rename Columns

In [37]:
sy_data = sy_data.rename(columns={'ID': 'individualID',
                                  'Taxa':'scientificName',
                                  'Measurements (NAME;NAME;NAME...) divided by ;':'measurementType',
                                  'Values (mm;mm;mm...) divided by ;':'measurementValue',
                                  'Gender': 'sex',
                                  'Age appearance':'lifeStage'})

Clean up lifeStage column

In [38]:
# Create Filters
adult = sy_data['lifeStage']=="ADL"
juvenile = sy_data['lifeStage']=="JUV"

# Assign correct terms using filters
sy_data['lifeStage'][adult] = "adult"
sy_data['lifeStage'][juvenile] = "juvenile"

<ipython-input-38-b12a5b877cf9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['lifeStage'][adult] = "adult"
<ipython-input-38-b12a5b877cf9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['lifeStage'][juvenile] = "juvenile"


Clean up sex column

In [39]:
male = sy_data['sex']=="M"
sy_data["sex"][male==True]="male"
sy_data['sex'][male==False]=""

<ipython-input-39-52260372d541>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data["sex"][male==True]="male"
<ipython-input-39-52260372d541>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['sex'][male==False]=""


Check is length of measurementType and measurementValue are the same 

In [46]:
def matching_size(row):
    """function to match size of measurement type and value"""
    mtype = row["measurementType"].split(';')
    mval = row["measurementValue"].split(';')
    if len(mtype) != len(mval):
        print ("Error: Size Mismatch")
        print(row.name)
        
sy_data.apply(matching_size,  axis =1)


Error: Size Mismatch
4
Error: Size Mismatch
7


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

Expand data using ; as delimiter

In [47]:
# Create a list of column names
column_names = list(sy_data.columns)

# Remove measurementType and measurementValue
column_names = column_names[: len(column_names) - 2] 

# Expand dataframe based off of delimeter
sy_data = sy_data.concat([Series(row[column_names], row['measurementType','measurementValue'].split(';'))              
                            for _, row in a.iterrows()]).reset_index()


AttributeError: 'DataFrame' object has no attribute 'concat'

In [3]:
#TODO: JSON specific columns